In [14]:
import random
import copy
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
candidate = {
    1: "Jackie Kasabach",
    2: "Jack Johnson",
    3: "Adam Frisch",
    4: "Torre",
    5: "Michael Behrendt",
    6: "Jason Lasser",
    7: "Michael Wampler",
    8: "Derek Johnson",
    9: "Brian D. Speck",
    10: "Write In 1",
    11: "Write In 2"
}

In [16]:
def load(path): 
    profile = []

    with open(path, 'r') as file:
        for line in file:
            line = line.strip()

            # for each line, split the votes and preferences
            if line:
                votes_str, preferences_str = line.split(':')
                votes = int(votes_str)

                preferences = []
                tied_preferences = []
                tie = False

                # add the votes to the first position of the profile
                preferences.append(votes)

                # add the preferences to the profile
                # split the tied votes and add them as a list to the profile
                for pref in preferences_str.split(','):
                    pref = pref.strip()

                    if "}" in pref:
                        pref = pref.replace('}', '')
                        tied_preferences.append(int(pref))
                        preferences.append(tied_preferences)
                        tie = False

                    elif "{" in pref or tie:
                        pref = pref.replace('{', '')
                        tied_preferences.append(int(pref))
                        tie = True
                    else:
                        preferences.append(int(pref))

                    # add the votes and preferences to the profile
                profile.append(preferences)

    return profile

# load data and convert
file_path = 'data.txt'
original_profile = load(file_path)
profile_length = len(original_profile)

In [17]:
def single_transferable_vote(profile, candidates, print_status=False):
    eliminated_candidates = []

    round = 0
    while len(eliminated_candidates) < len(candidates):

        # reset the round
        round += 1
        tally = [0] * len(candidates)
        for index in eliminated_candidates:
            tally[index] = None
        eliminate = []

        # tally the votes for each candidate
        for pref in profile:

            n = 1
            # search for the candidate with the highest preference that has not been eliminated
            searching = True
            while searching:

                # if the preference is a tie 
                if isinstance(pref[n], list):
                    m = 0
                    # for each tied preference, check if the candidate has been eliminated otherwise tally the votes
                    for _ in pref[n]:

                        if pref[n][m] - 1 not in eliminated_candidates:
                            candidate = pref[n][m] - 1
                            tally[candidate] += pref[0]

                        m += 1
                        if m >= len(pref[n]):
                            searching = False

                # check if the candidate has been eliminated
                elif pref[n] - 1 not in eliminated_candidates:
                    candidate = pref[n] - 1
                    tally[candidate] += pref[0]
                    searching = False
                n += 1
                if n >= len(pref):
                    searching = False

        # eliminate the candidate with the lowest votes
        lowest_votes = min(value for value in tally if value is not None)
        eliminate = [i for i, value in enumerate(tally) if value == lowest_votes]
        eliminated_candidates.extend(eliminate)

        # find the winner
        highest_votes = max(value for value in tally if value is not None)
        winner = [i for i, value in enumerate(tally) if value == highest_votes]

        if print_status:
            print("---ROUND {}---".format(round))
            print("Tally: {}".format(tally))
            print("Eliminated {}:".format(eliminate))
            print()

    return winner, highest_votes

In [18]:
profile_length = len(original_profile)

# FIND THE WINNER
original_results = single_transferable_vote(original_profile, candidate, print_status=True)
for result in original_results[0]:
    print("{} wins with {} votes! \n".format(candidate[result + 1], original_results[1]))

---ROUND 1---
Tally: [247, 456, 415, 385, 352, 27, 94, 463, 38, 0, 0]
Eliminated [9, 10]:

---ROUND 2---
Tally: [247, 456, 415, 385, 352, 27, 94, 463, 38, None, None]
Eliminated [5]:

---ROUND 3---
Tally: [247, 460, 416, 394, 355, None, 96, 466, 41, None, None]
Eliminated [8]:

---ROUND 4---
Tally: [253, 464, 422, 396, 365, None, 100, 474, None, None, None]
Eliminated [6]:

---ROUND 5---
Tally: [259, 478, 433, 416, 383, None, None, 494, None, None, None]
Eliminated [0]:

---ROUND 6---
Tally: [None, 535, 467, 458, 445, None, None, 547, None, None, None]
Eliminated [4]:

---ROUND 7---
Tally: [None, 620, 557, 552, None, None, None, 671, None, None, None]
Eliminated [3]:

---ROUND 8---
Tally: [None, 801, 672, None, None, None, None, 815, None, None, None]
Eliminated [2]:

---ROUND 9---
Tally: [None, 908, None, None, None, None, None, 1227, None, None, None]
Eliminated [1]:

---ROUND 10---
Tally: [None, None, None, None, None, None, None, 1685, None, None, None]
Eliminated [7]:

Derek Johns

In [19]:
def manipulate(profile):

    saved_profile = None
    saved_pref = None
    saved_index = None
    saved_n = 1000000
    saved_original_pref = None

    for pref in profile:

        index = profile.index(pref)
        pref_copy = copy.deepcopy(pref)
    
        i = 1
        for alt in pref[1:]:

            profile_copy = copy.deepcopy(profile)

            if isinstance(alt, list):
                for a in alt:
                    alt.remove(a)
                    pref_copy.insert(1, a)
                    profile_copy[index] = pref_copy
            else:
                pref_copy.pop(i)
                pref_copy.insert(1, alt)
                profile_copy[index] = pref_copy

            election = single_transferable_vote(profile_copy, candidate, print_status=False)
               
            if election[0][0] != original_results[0][0] and pref[0] < saved_n:
                saved_profile = copy.deepcopy(profile_copy)
                saved_pref = copy.deepcopy(pref_copy)
                saved_original_pref = copy.deepcopy(pref)
                saved_index = index
                saved_n = pref[0]
                
            i += 1
            
    return saved_profile, saved_index, saved_pref, saved_n, saved_original_pref

final_profile, final_index, final_pref, final_n, final_origianl_pref = manipulate(original_profile)

In [20]:
# FIND MANIPULATED WINNER
manipulated_results = single_transferable_vote(final_profile, candidate, print_status=True)
for result in manipulated_results[0]:
    print("{} wins with {} votes! \n".format(candidate[result + 1], manipulated_results[1]))

print("INDEX: ", final_index)
print("PREF: ", final_pref)
print("N: ", final_n)
print("ORIGINAL PREF: ", final_origianl_pref)

TypeError: 'NoneType' object is not iterable